# 웹에서 분석 데이터 가져오기

### 🔰 셀프 주유소가 정말 저렴할까?

- 서울시에 등록된 주유소의 가격 정보를 확인할 수 있는 Opinet 사이트 활용

- 데이터 확보를 위한 작업

    1. 사이트 구조 확인 : https://www.opinet.co.kr/user/main/mainView.do

    2. 목표 데이터 설정 : https://www.opinet.co.kr/searRgSelect.do

        - 브랜드 
        - 가격 
        - 셀프 주유 여부 
        - 위치 

### 🔰 웹 데이터 가져오기 by selenium

- Chrome 제어창 띄우기

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

chrome_driver_path = "../chromedriver-win64/chromedriver.exe"
service = Service(executable_path=chrome_driver_path)
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=options, service=service)

# Opinet > 싼 주유소 찾기 > 지역별 페이지로 접근
url = "https://www.opinet.co.kr/searRgSelect.do"
driver.get(url)

In [2]:
driver.set_window_position(x=3268, y=235)
driver.set_window_size(width=1000, height=1080)

-----

- 문제 : 팝업창

	- 만약 팝업창이 뜨는 사이트라면 해당 URL로 한 번에 접근이 안된다.

	- 원하는 페이지로 접근하지 못하고 메인페이지로 접속이 되면서 팝업창이 뜨는 문제가 발생한다.

In [ ]:
# 1. 팝업창으로 화면 전환 후 닫다준다.
driver.switch_to.window(driver.window_handles[-1])
driver.close()

In [ ]:
# 2. 메인창으로 화면 전환 후 접근 페이지를 다시 요청한다.
driver.switch_to.window(driver.window_handles[-1])
driver.get(url)

In [ ]:
# 문제 해결 함수 : 요청 페이지 가져오기

import time

def request_page_get():
    # 요청 페이지 최초 접근
    chrome_driver_path = "../chromedriver-win64/chromedriver.exe"
    options = webdriver.ChromeOptions()
    service = Service(chrome_driver_path)
    driver = webdriver.Chrome(options=options, service=service)
    url = "https://www.opinet.co.kr/searRgSelect.do"
    driver.get(url)
    time.sleep(3) # selenium의 속도가 빠르지 못해 다음 수행에서 오류가 날 수 있다.
    
    # 팝업창으로 전환
    driver.switch_to.window(driver.window_handles[-1])
    
    # 팝업창 닫기
    driver.close()
    time.sleep(3)
    
    # 메인화면 창으로 전환
    driver.switch_to.window(driver.window_handles[-1])
    
    # 접근 URL 다시 요청
    driver.get(url)

In [ ]:
request_page_get()

-----

In [3]:
from selenium.webdriver.common.by import By

- 지역: 시/도 데이터

	- 토글로 되어있어 '서울'을 선택해야 한다.

	- `id="SIDO_NM0"` 속성으로 접근한다.

![오피넷_SIDO](https://github.com/ElaYJ/Study_EDA/assets/153154981/046c72ba-b406-472a-9fb0-d713eefbd825)

In [4]:
sido_list_raw = driver.find_element(By.ID, "SIDO_NM0")
sido_list_raw, sido_list_raw.text

(<selenium.webdriver.remote.webelement.WebElement (session="1d47ec006fe13799709ac817fc72eb6d", element="BA465787FCD7D3279E07D95CF63C6DE9_element_2323")>,
 '            시/도\n            \n             \n             \n              서울\n             \n            \n             \n             \n              부산\n             \n            \n             \n             \n              대구\n             \n            \n             \n             \n              인천\n             \n            \n             \n             \n              광주\n             \n            \n             \n             \n              대전\n             \n            \n             \n             \n              울산\n             \n            \n             \n             \n              세종\n             \n            \n             \n              경기\n             \n             \n            \n             \n             \n              강원\n             \n            \n             \n             \n             

In [5]:
print(sido_list_raw.text)

            시/도
            
             
             
              서울
             
            
             
             
              부산
             
            
             
             
              대구
             
            
             
             
              인천
             
            
             
             
              광주
             
            
             
             
              대전
             
            
             
             
              울산
             
            
             
             
              세종
             
            
             
              경기
             
             
            
             
             
              강원
             
            
             
             
              충북
             
            
             
             
              충남
             
            
             
             
              전북
             
            
             
             
       

In [8]:
sido_list = sido_list_raw.find_elements(By.TAG_NAME, "option")
len(sido_list), sido_list[0].text, sido_list[17].text

(18, '시/도', '제주')

- `get_attribute()`

	- `get_attribute("value")` : value= 속성값 가져오기

In [9]:
sido_list[1].get_attribute("value")

'서울특별시'

In [10]:
# means_1.

sido_names = []
for option in sido_list:
    sido_names.append(option.get_attribute("value"))
    
sido_names

['',
 '서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [11]:
# means_2.

sido_names = [option.get_attribute("value") for option in sido_list]
sido_names[:5]

['', '서울특별시', '부산광역시', '대구광역시', '인천광역시']

In [12]:
sido_names = sido_names[1:] # del sido_names[0]
sido_names

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원특별자치도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [13]:
sido_names[0]

'서울특별시'

- 지역 > 시/도 > 서울 선택

In [14]:
sido_list_raw.send_keys(sido_names[0])

- 지역: 구 데이터

In [15]:
# 부모 태그: <select> tag
gu_list_raw = driver.find_element(By.ID, "SIGUNGU_NM0")

# 자식 태그: <option> tag
gu_list = gu_list_raw.find_elements(By.TAG_NAME, "option")

# 태그 속성값: <option> tag의 value= 값
gu_names = [option.get_attribute("value") for option in gu_list]

del gu_names[0] # gu_names = gu_names[1:]
gu_names[:5], len(gu_names)

(['강남구', '강동구', '강북구', '강서구', '관악구'], 25)

- 지역 > 시/군/구 > 성동구 선택

In [16]:
gu_list_raw.send_keys(gu_names[15])

![04_OP_selenium_exel_save](https://github.com/ElaYJ/Study_EDA/assets/153154981/e40bd01f-a80c-4d6d-9dfc-56546218c715)

### 🔰 엑셀 저장

- 구별 주유소 정보를 엑셀로 저장

In [17]:
# means_1. selector
driver.find_element(By.CSS_SELECTOR, "#glopopd_excel").click()

In [18]:
# means_2. xpath
driver.find_element(By.XPATH, '//*[@id="glopopd_excel"]').click()

In [19]:
# means_3. id
driver.find_element(By.ID, "glopopd_excel").click()

- `progressbar` 모듈

	- vscode에서는 tqdm 모듈을 사용하지 못해 비슷한 기능의 다른 모듈 사용

In [22]:
import progressbar
import time

# 진행 상황 표시 위젯 생성
bar = progressbar.ProgressBar(
	maxval=100,
 	widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()]
)

# 반복 작업 예시 (0부터 99까지 반복)
for i in bar(range(100)):
    # 작업 수행
    time.sleep(0.01)

[========================================================================] 100%


In [24]:
import time
# jupyter notebook
# from tqdm import tqdm_notebook

# vscode
from progressbar import ProgressBar, Bar, Percentage
bar = ProgressBar(maxval=100, widgets=[Bar('=', '[', ']'), " ", Percentage()])


# jupyter notebook
# for gu in tqdm_notebook(gu_names):

# vscode
for gu in bar(gu_names):
    element = driver.find_element(By.ID, "SIGUNGU_NM0")
    element.send_keys(gu)
    time.sleep(3)
    
    element_get_excel = driver.find_element(By.ID, "glopopd_excel")
    element_get_excel.click()
    time.sleep(3)

[========================================================================] 100%


In [25]:
driver.close()